In [1]:
## required imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
## read data
df_1 = pd.read_csv('Twitter_5_year.csv')

In [3]:
df_other_artists = pd.read_csv('SZA_Chainsmoker_Twitter.csv')

In [4]:
df_new_artists = pd.read_csv('Twitter_5_year_new_artists.csv')

In [5]:
df = pd.concat([df_1,df_other_artists,df_new_artists])

In [6]:
# Shape
df.shape 

(351806, 15)

In [7]:
# Print columns
df.columns

Index(['Account', 'User Name', 'Followers at Posting', 'Created', 'Type',
       'Likes', 'Retweets', 'URL', 'Message', 'Screen Name', 'Link 1',
       'Final Link 1', 'Link 2', 'Final Link 2', 'Score'],
      dtype='object')

In [8]:
df.nunique()

Account                    148
User Name                  148
Followers at Posting     95963
Created                 350077
Type                         6
Likes                    33662
Retweets                 18339
URL                     351806
Message                 347186
Screen Name                162
Link 1                  260434
Final Link 1            258077
Link 2                   34489
Final Link 2             34073
Score                     7868
dtype: int64

In [9]:
df['Type'].value_counts()

Link            171850
Tweet            72616
Photo            69085
Native Video     26629
Video            11555
Vine                71
Name: Type, dtype: int64

In [10]:
df.count()

Account                 351806
User Name               351806
Followers at Posting    338345
Created                 351806
Type                    351806
Likes                   351806
Retweets                351806
URL                     351806
Message                 351806
Screen Name             345077
Link 1                  279215
Final Link 1            279215
Link 2                   42399
Final Link 2             42399
Score                   351806
dtype: int64

In [11]:
sum(df['Followers at Posting'].isna())

13461

In [12]:
df_selected = df[['User Name', 'Followers at Posting', 'Created', 'Type',
       'Likes', 'Retweets','Message','Score']]
df_selected.head()

,User Name,Followers at Posting,Created,Type,Likes,Retweets,Message,Score
0,therealjuicyj,2645413.0,2019-06-13 23:49:25 EDT,Tweet,864,190,Whomever made bets made good money tonight,-3.59
1,lukecombs,336828.0,2019-06-13 23:18:23 EDT,Tweet,3859,102,.@nicohocking is bae,1.14
2,adamlambert,2477885.0,2019-06-13 23:03:07 EDT,Link,832,343,Korea! Tickets are on sale NOW for Queen + Ada...,-1.33
3,russdiemon,2043124.0,2019-06-13 22:56:16 EDT,Tweet,986,31,What questions y’all want @KidSuper to ask me?...,-9.22
4,gerardoortiznet,3082321.0,2019-06-13 22:43:58 EDT,Photo,727,76,🚘😎😜 https://t.co/4yeXAIbwA7,1.89


In [13]:
df_selected.count()

User Name               351806
Followers at Posting    338345
Created                 351806
Type                    351806
Likes                   351806
Retweets                351806
Message                 351806
Score                   351806
dtype: int64

In [14]:
df_selected = df_selected.dropna()
df_selected = df_selected.reset_index(drop = True)

In [15]:
df_selected.count()

User Name               338345
Followers at Posting    338345
Created                 338345
Type                    338345
Likes                   338345
Retweets                338345
Message                 338345
Score                   338345
dtype: int64

In [16]:
# Convert "Created" into datetime 
df_selected['Created'] = pd.to_datetime(df_selected['Created'])

In [17]:
#Number of characters in message custom feature
df_selected['message_length'] = df_selected['Message'].apply(len)

In [18]:
#Number of hashtags custom feature
df_selected['hashtag_count'] = df_selected.apply(lambda x: x['Message'].count('#'), axis=1)

In [19]:
# Combine post types
df_selected['Type'].value_counts()

Link            166131
Tweet            68342
Photo            66959
Native Video     26046
Video            10816
Vine                51
Name: Type, dtype: int64

In [20]:
df_selected.loc[df_selected['Type'] == 'Vine', 'Type'] = 'Video'
df_selected.loc[df_selected['Type'] == 'Native Video', 'Type'] = 'Video'

In [21]:
df_selected['Type'].value_counts()

Link     166131
Tweet     68342
Photo     66959
Video     36913
Name: Type, dtype: int64

In [22]:
#OHE for Type Field
df_selected = pd.concat([df_selected,pd.get_dummies(df_selected['Type'],prefix='type')],axis=1)

In [23]:
def time_of_day_creation(x):
    '''
    Takes in an integer and returns one of four time slots:
    ['12am-6am', '6am-12pm', '12pm-6pm', '6pm-12am']
    '''
    
    slots_out = ['12am-6am', '6am-12pm', '12pm-6pm', '6pm-12am']
    slot0 = np.arange(0,6)
    slot1 = np.arange(6,12)
    slot2 = np.arange(12,18)
    slot3 = np.arange(18,24)

    if x in slot0:
        return slots_out[0]
    elif x in slot1:
        return slots_out[1]
    elif x in slot2:
        return slots_out[2]
    elif x in slot3:
        return slots_out[3]
    else:
        return 'NaN'

In [24]:
#Creating time of day buckets (e.g. 0-6am, 6-12pm, 12-6, 6-12am)
df_selected['time_of_day'] = df_selected['Created'].apply(lambda x : time_of_day_creation(x.hour))

In [25]:
#OHE for time of day field
df_selected = pd.concat([df_selected,pd.get_dummies(df_selected['time_of_day'],prefix='tod')],axis=1)

In [26]:
df_selected.sort_values(by=['User Name','Created'], inplace=True)

In [27]:
#Time between prior and current post
df_selected['time_since_last_post'] = df_selected.groupby('User Name')['Created'].diff()

In [28]:
df_selected['within_week_release'] = 0

In [29]:
df_selected['within_month_release'] = 0

In [30]:
df_selected['social_engagement_score'] = df_selected.apply(lambda x: (x['Likes'] + x['Retweets']) / x['Followers at Posting'], axis=1)

In [31]:
df_selected['social_engagement_score'].describe()

count    338345.000000
mean          0.002511
std           0.011393
min           0.000000
25%           0.000077
50%           0.000323
75%           0.001494
max           1.525838
Name: social_engagement_score, dtype: float64

In [32]:
df_selected = df_selected.drop(['Type','Likes', 'Retweets','time_of_day'], axis=1)

In [33]:
df_selected.reset_index(drop=True, inplace=True)

In [34]:
df_selected.head()

,User Name,Followers at Posting,Created,Message,Score,message_length,hashtag_count,type_Link,type_Photo,type_Tweet,type_Video,tod_12am-6am,tod_12pm-6pm,tod_6am-12pm,tod_6pm-12am,time_since_last_post,within_week_release,within_month_release,social_engagement_score
0,1future,1668274.0,2015-09-11 19:24:44,Download 'March Madness' on iTunes NOW!!!! htt...,-3.3,70,1,1,0,0,0,0,0,0,1,NaT,0,0,0.000225
1,1future,1668274.0,2015-09-11 21:42:00,Naw frfr... https://t.co/Dmav5zl5N4,-3.88,35,0,1,0,0,0,0,0,0,1,02:17:16,0,0,0.000191
2,1future,1668274.0,2015-09-11 22:33:50,Stream #DS2 on @Spotify NOW!!!!! http://t.co/V...,-3.14,69,2,1,0,0,0,0,0,0,1,00:51:50,0,0,0.000236
3,1future,1668274.0,2015-09-12 01:14:14,R/p @theshootrr https://t.co/emi7dj17hH,-7.37,39,0,1,0,0,0,1,0,0,0,02:40:24,0,0,0.000101
4,1future,1668274.0,2015-09-12 03:29:25,Took the stage in Toronto & shit will never be...,1.17,124,1,0,0,1,0,1,0,0,0,02:15:11,0,0,0.003585


In [35]:
df_selected.columns

Index(['User Name', 'Followers at Posting', 'Created', 'Message', 'Score',
       'message_length', 'hashtag_count', 'type_Link', 'type_Photo',
       'type_Tweet', 'type_Video', 'tod_12am-6am', 'tod_12pm-6pm',
       'tod_6am-12pm', 'tod_6pm-12am', 'time_since_last_post',
       'within_week_release', 'within_month_release',
       'social_engagement_score'],
      dtype='object')

In [36]:
# Rename columns
df_selected.rename(columns={"Created": "created",
                            "Message": "description",
                            "Score":"crowdtangle_score", 
                            "Followers at Posting" : "count_of_followers",
                            "tod_12am-6am" : "created_12am-6am",
                            "tod_6am-12pm" : "created_6am-12pm",
                            "tod_12pm-6pm" : "created_12pm-6pm",
                            "tod_6pm-12am" : "created_6pm-12am",
                            "User Name" : "Twitter_Handle",
                            "message_length":"description_length",
                            "type_Photo":"type_photo",
                            "type_Tweet":"type_text",
                            "type_Video":"type_video",
                            "type_Link":"type_link"
                           }, inplace=True)

In [37]:
df_selected.head()

,Twitter_Handle,count_of_followers,created,description,crowdtangle_score,description_length,hashtag_count,type_link,type_photo,type_text,type_video,created_12am-6am,created_12pm-6pm,created_6am-12pm,created_6pm-12am,time_since_last_post,within_week_release,within_month_release,social_engagement_score
0,1future,1668274.0,2015-09-11 19:24:44,Download 'March Madness' on iTunes NOW!!!! htt...,-3.3,70,1,1,0,0,0,0,0,0,1,NaT,0,0,0.000225
1,1future,1668274.0,2015-09-11 21:42:00,Naw frfr... https://t.co/Dmav5zl5N4,-3.88,35,0,1,0,0,0,0,0,0,1,02:17:16,0,0,0.000191
2,1future,1668274.0,2015-09-11 22:33:50,Stream #DS2 on @Spotify NOW!!!!! http://t.co/V...,-3.14,69,2,1,0,0,0,0,0,0,1,00:51:50,0,0,0.000236
3,1future,1668274.0,2015-09-12 01:14:14,R/p @theshootrr https://t.co/emi7dj17hH,-7.37,39,0,1,0,0,0,1,0,0,0,02:40:24,0,0,0.000101
4,1future,1668274.0,2015-09-12 03:29:25,Took the stage in Toronto & shit will never be...,1.17,124,1,0,0,1,0,1,0,0,0,02:15:11,0,0,0.003585


In [38]:
# To be replaced with below function
#df_selected['posts_last_7days'] = 0
#df_selected['posts_last_30days'] = 0

In [39]:
def calculateRollupPosts(social_media_df,social_media_type, social_media_handle_field):
    time_range = pd.date_range(start='1/1/2016', end='6/30/2019', freq='d').to_frame(index=False)
    time_range.rename(columns={0: 'Time Period'}, inplace=True)
    master_artists_list = pd.read_csv('master_artists_list.csv', usecols=[social_media_type])
    time_range['join_key'] = 0
    master_artists_list['join_key'] = 0
    artists_time_range = master_artists_list.merge(time_range, how='left', on = 'join_key')
    artists_time_range.drop('join_key',1, inplace=True)
    artists_time_range.rename(columns={0: 'Time Period'}, inplace=True)
    social_media_df['Created_Day'] = social_media_df.created.dt.floor('d')
    df_aggregate = social_media_df.groupby([social_media_handle_field, 'Created_Day'], as_index=False).agg({"created": "count"})
    df_aggregate.rename(columns={'created': 'Posts In Day'}, inplace=True)
    df_merged = artists_time_range.merge(df_aggregate, how='left', left_on=[social_media_type,'Time Period'], right_on=[social_media_handle_field,'Created_Day'])
    df_merged['Posts In Day'] = df_merged['Posts In Day'].fillna(0)
    df_merged['last_7_days'] = df_merged[['Posts In Day']].groupby(df_merged[social_media_type]).apply(lambda g: g.rolling(7, min_periods=1).sum())
    df_merged['last_30_days'] = df_merged[['Posts In Day']].groupby(df_merged[social_media_type]).apply(lambda g: g.rolling(30, min_periods=1).sum())
    df_merged.drop(columns=[social_media_handle_field, 'Created_Day','Posts In Day'], inplace=True)
    social_media_df = social_media_df.merge(df_merged, how='left', right_on=[social_media_type,'Time Period'], left_on=[social_media_handle_field,'Created_Day'])
    social_media_df.drop(columns=[social_media_type, 'Time Period','Created_Day'], inplace=True)
    
    return social_media_df

In [40]:
df_selected = calculateRollupPosts(df_selected,'twitter','Twitter_Handle')

In [42]:
#Removing the 7 artists
artists_to_remove = ['carlosvives', 'mirandalambert', 'BradPaisley', 'TheTimMcGraw',
                     'ChrisYoungMusic', 'lukecombs', 'WillieNelson']
df_selected = df_selected[~df_selected['Twitter_Handle'].isin(artists_to_remove)]

In [43]:
df_selected.to_csv('twitter_features.csv')